# WEB+DB PRESS vol.118 Pythonデータ可視化入門　第3章～第5章

## 参考記事  

- Pythonデータ可視化入門
- 著者：小川英幸さん@ogawahideyuki
- ソースコード：https://github.com/mazarimono/webdb_notebook/

## 対応章

- 第５章　ブラウザで動くダッシュボードの作成

# ブラウザで動くダッシュボードの作成

- コンポーネントと呼ばれるツールを組み合わせてDashアプリケーションを作成
- これらをうまく組み合わせる事で、インタラクティブなアプリケーションを作成可能

## コンポーネントを構成するパッケージ

|コンポーネントの種類|概要|
|--------------------|----|
|Dash HTML Components|HTML要素を作成する|
|Dash Core Components|スライダ、ドロップダウン、グラフなどを作成する|
|Dash Data Table|インタラクティブなテーブルを作成する|
|Dash Bio|バイオインフォマティクス向けグラフを作成する|
|Dash DAQ|IoT端末などからデータ収集用ツールを提供する|
|Dash Canvas|画像を処理する|
|Dash Cytoscape|ネットワークを可視化する|

In [ ]:
# JupyterDashをGoogle Colaboratoryで使える様にする
!pip install jupyter_dash

     |████████████████████████████████| 51kB 1.7MB/s eta 0:00:011
     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 1.0MB 25.7MB/s 
     |████████████████████████████████| 3.5MB 31.7MB/s 
     |████████████████████████████████| 194kB 31.0MB/s 
     |████████████████████████████████| 1.8MB 34.7MB/s 
     |████████████████████████████████| 358kB 37.8MB/s 
  Created wheel for ansi2html: filename=ansi2html-1.5.2-cp36-none-any.whl size=13862 sha256=753a0edf383a2c806f8ffa82c9da139d94e7652031b59ae489ce196de2d83a4d
  Stored in directory: /root/.cache/pip/wheels/f9/ea/e0/1ed3c3945ce073f930f3e5aa2b9bbadb03cd2efc8f8695cd61
  Created wheel for dash: filename=dash-1.17.0-cp36-none-any.whl size=83728 sha256=f220b4dea39379b071463f2d1e769d2dc42d1d7f6545e6074c47403b34ad2852
  Stored in directory: /root/.cache/pip/wheels/e0/32/13/8a569b638a79cf80a4a721f56040d91198adec28e3bc8123fc
  Created wheel for flask-compress: filename=Flask_Compress-1.8.0-cp36-none-any.w

## HTML要素を作成するパッケージ：Dash HTML Components

- Dash HTML ComponentsはすべてのHTMLタグをクラスとして提供するパッケージ
- HTMLタグの最初の文字を大文字にしたクラスを提供

In [ ]:
# Hello World アプリケーションの作例

from jupyter_dash import JupyterDash
import dash_html_components as html

app = JupyterDash(__name__) # JupyterDashインスタンスを作成
app.layout = html.H1('HELLO WEB+DB PRESS')
app.run_server(mode='inline') # flaskサーバを起動(通常は8050ポート起動)

In [ ]:
# 複数コンポーネントを組み合わせた作例

app = JupyterDash(__name__)
app.layout = html.Div([
  html.P('Show Image', style={'backgroundColor': 'red', 'color': 'white'}),
  html.Img(src='https://3.bp.blogspot.com/--PL6hlrPIPo/Uku_GiUXI6I/AAAAAAAAYuI/aOQxoDd6Oy8/s400/kankou_kinkakuji.png')
])
app.run_server(mode='inline')

## ツールを提供するパッケージ：Dash Core Components

- ドロップダウンやグラフなどを提供

In [ ]:
# Graph コンポーネント

import dash_core_components as dcc
import pandas as pd
import plotly.express as px

df = pd.read_csv('kakei_data.csv', index_col=0, parse_dates=['date'])
app = JupyterDash(__name__)
app.layout = html.Div([
  html.H1('Graphコンポーネントの利用'),
  dcc.Graph(
      figure=px.line(
          df, x='date', y='スポーツ', title='スポーツ支出'
      )
  )
])
app.run_server(mode='inline')

In [ ]:
# Markdown コンポーネント

app = JupyterDash(__name__)

app.layout = dcc.Markdown('''
  # Markdownを使います
  ## Dash Components
  ### dash html Components
  - htmlタグを提供
  ### dash core Components
  - 様々なツールを提供
''', style={
    'color': 'green', 'backgroundColor': 'black', 'padding': '2%'
})

app.run_server(mode='inline')

In [ ]:
# 選択 / 入力コンポーネント

from datetime import date

app = JupyterDash(__name__)
four_styles = {
    'margin': '3%', 
    'width': '40%', 
    'display': 'inline-block', # コンポーネントを横に並べたいため
    'verticalAlign': 'top' # コンポーネントの高さを合わせるため
}
selection = {'tokyo', 'osaka', 'fukuoka'}

app.layout = html.Div([
    html.Div([dcc.Dropdown(options=[{'value': i, 'label': i} for i in selection], value='tokyo')], style=four_styles),
    html.Div([dcc.Input(placeholder='文字を入力してください')], style=four_styles),
    html.Div([dcc.DatePickerRange(start_date=date(2017, 12, 18), end_date=date(2020, 6, 22),),], style=four_styles),
    html.Div([html.Button('HELLO PUSH ME')], style=four_styles)
])

app.run_server(mode='inline')

## Dashにインタラクティブ要素を与える「コールバック」

- インタラクティブ要素を与える機能

In [ ]:
# コールバックの基本的な作り方

from dash.dependencies import Input, Output, State
app = JupyterDash(__name__)

graph_type_list = [px.line, px.scatter, px.bar]

app.layout = html.Div([
    dcc.Dropdown(
        id = 'my_dropdown',
        options = [{'label': type_.__name__, 'value': num}
            for num, type_ in enumerate(graph_type_list)], 
            value = 0),
    html.Button(
        id='my_button',
        children='Update Graph' # この場合はボタン名になる
    ),
    dcc.Graph(
        id='my_graph',
    ),
])

# app.callbackデコレータを作成
# - Output/Input/Stateに指定するキー名は、描画するHTMLのidに紐づく必要有
# - Outputは複数であればリストを設定する必要あるが、InputとStateは単数でもリストを指定する必要有
@app.callback(Output('my_graph', 'figure'), # Output:コールバック関数の戻り値の出力先指定
              [Input('my_button', 'n_clicks')], # Input:呼び出しを指定
              [State('my_dropdown', 'value')]) # State:状態をコンポーネント保持するコンポーネント
def update_graph(n_clicks, selected_value): # 関数名と引数名はなんでもいい
  return graph_type_list[selected_value](x=[1,2,3,4,5], y=[1,2,3,4,5])

app.run_server(mode='inline')

<IPython.core.display.Javascript object>